# Reinforcement Learning
### Policy iteration

The **Policy Iteration** is another model-based method used in **Reinforcement Learning** (RL) to obtain the optimal policy for a **Markov Decision Process** (MDP). The algorithm starts with a random policy. Then, it evaluates its value function called **policy evaluation**. After that, we improve the policy greedily called **policy improvement**. The process is repeated until convergence to the optimal policy. 

<br>The key steps in **Policy Iteration**:
1. **Initialize** a random policy $\pi$. Start with an arbitrary value function $v(s)$ for all states $s$.
2. Repeat until policy converges:
<br>a. **Policy Evaluation**:  
<br>Compute $v_{\pi(s)}$ for all states using iterative Bellman updates until it converges:
<br>$v_{\pi}(s)=\sum_{s′} p(s′|s,\pi(s))[r(s,\pi(s),s′)+\gamma v_{\pi}(s′)]$
<br>b. **Policy Improvement**:  
<br>Update $\pi(s) = argmax_a \sum_{s'} p(s'|s,a) [r(s,a,s') + \gamma v_{\pi}(s')]$.  
If policy $\pi$ does not change, stop.  
3. Return optimal $v$ and $\pi$.

<hr>

In the example in this Notebook, we use a **Grid World** environment (similar to the one we used in value iteration):
 - **States:** A 3x3 grid (9 states), labeled as (0,0) to (2,2).
 - **Actions:** Up, Down, Left, Right.
 - **Rewards:**
    - Reaching the goal state (2,2) gives a reward of +10.
    - Reaching a "pit" state (1,1) gives a reward of −10.
    - All other transitions give a reward of −1.
- **Terminal States:** (2,2) (goal) and (1,1) (pit).
- **Transition Probabilities:**
    - Moving in the intended direction succeeds with probability 0.8.
    - With probability 0.2, the agent moves in a random direction

<hr>
https://github.com/ostad-ai/Reinforcement-Learning
<br> Explanation: https://www.pinterest.com/HamedShahHosseini/Reinforcement-Learning

In [1]:
import random
# Define the grid world
states = [(i, j) for i in range(3) for j in range(3)]  # 3x3 grid
actions = ["up", "down", "left", "right"]              # Possible actions
gamma = 0.9                                           # Discount factor
theta = 1e-6                                          # Convergence threshold

# Terminal states
terminal_states = {(2, 2): 10, (1, 1): -10}  # (state: reward)

# Transition probabilities
def transition_probability(s, a, s_):
    if s in terminal_states:  # Terminal states have no transitions
        return 0
    intended_s = get_intended_state(s, a)
    if s_ == intended_s:
        return 0.8
    elif s_ in get_neighbors(s):
        return 0.2 / len(get_neighbors(s))
    else:
        return 0

# Reward function
def reward(s, a, s_):
    if s_ in terminal_states:
        return terminal_states[s_]
    return -1  # Default reward for non-terminal transitions

# The state the agent goes to, 
# from current state s, by taking action a
# if everything was deterministic
def get_intended_state(s, a):
    i, j = s
    if a == "up":
        return (max(i - 1, 0), j)
    elif a == "down":
        return (min(i + 1, 2), j)
    elif a == "left":
        return (i, max(j - 1, 0))
    elif a == "right":
        return (i, min(j + 1, 2))

# Immediate neighbors of the current state s 
def get_neighbors(s):
    i, j = s
    neighbors = []
    for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        ni, nj = i + di, j + dj
        if 0 <= ni < 3 and 0 <= nj < 3:
            neighbors.append((ni, nj))
    return neighbors

def policy_iteration(states, actions, P, R, gamma=0.9, theta=1e-6):
    # 1. Initialize random policy
    policy = {s: random.choice(actions) for s in states}
    v = {s: 0 for s in states}
    
    while True:
        # 2. Policy Evaluation (Iterative Bellman updates for fixed policy)
        while True:
            delta = 0
            for s in states:
                if s in terminal_states:  # Skip terminal states
                    continue
                v_current = v[s]
                a = policy[s]
                v[s] = sum(P(s, a, s_) * (R(s, a, s_) + gamma * v[s_])
                       for s_ in states)
                delta = max(delta, abs(v_current - v[s]))
            if delta < theta:
                break
        
        # 3. Policy Improvement (Greedy update)
        policy_stable = True
        for s in states:
            if s in terminal_states:  # Skip terminal states
                policy[s]=None
                continue
            old_action = policy[s]
            policy[s] = max(actions, 
            key=lambda a: sum(P(s, a, s_) * (R(s, a, s_) + gamma * v[s_]) 
                                          for s_ in states))
            if old_action != policy[s]:
                policy_stable = False
        
        if policy_stable:
            return v, policy

In [3]:
# Run policy iteration
v,optimal_policy=policy_iteration(states,actions,
                                transition_probability,reward)
print('Environment: A 3*3 Grid World')
print('Algorithm: Policy Iteration')
print(30*'-')
print("Optimal Value Function v(s):")
for row in range(3):
    for col in range(3):
        print(f"State {(row,col)}: {v[(row,col)]:.2f}",end=', ')
    print('')
print("\nOptimal Policy \u03c0(s):")
for row in range(3):
    for col in range(3):
        print(f"State {(row,col)}: {optimal_policy[(row,col)]}",
              end=', ')
    print('')

Environment: A 3*3 Grid World
Algorithm: Policy Iteration
------------------------------
Optimal Value Function v(s):
State (0, 0): 0.63, State (0, 1): 1.89, State (0, 2): 4.71, 
State (1, 0): 1.89, State (1, 1): 0.00, State (1, 2): 7.55, 
State (2, 0): 4.71, State (2, 1): 7.55, State (2, 2): 0.00, 

Optimal Policy π(s):
State (0, 0): right, State (0, 1): right, State (0, 2): down, 
State (1, 0): down, State (1, 1): None, State (1, 2): down, 
State (2, 0): right, State (2, 1): right, State (2, 2): None, 
